In [27]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [28]:
df = pd.read_csv("diff_data/cems_diffs_isorto.csv", index_col='DATE_UTC')

In [29]:
# Convert to UTC (as opposed to UTC-5)
df.index = pd.DatetimeIndex(df.reset_index().DATE_UTC) + timedelta(hours=5)

# Remove columns with damage data 
df = df[df.columns[~df.columns.str.endswith('ap2')]]
df = df[df.columns[~df.columns.str.endswith('dam')]]
df = df[df.columns[~df.columns.str.endswith('eas')]]

In [30]:
# calculate MEF (assumed to be emissions / generation)
df['co2_mef'] = df['co2_kg']/df['gload_mwh']
df['so2_mef'] = df['so2_kg']/df['gload_mwh']
df['nox_mef'] = df['nox_kg']/df['gload_mwh']
df['pm25_mef'] = df['pm25_kg']/df['gload_mwh']
df = df[df.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
df = df.dropna() 

In [31]:
df.describe()

,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,co2_mef,so2_mef,nox_mef,pm25_mef
count,736057.000000,736057.000000,736057.000000,736057.000000,7.360570e+05,7.360570e+05,736057.000000,736057.000000,736057.000000
mean,0.114730,-0.439396,-0.118684,0.003811,5.577672e+01,5.912107e+02,0.649025,0.540102,0.069971
std,1467.368385,3195.501563,1085.100576,187.393820,9.780475e+05,3.870106e+04,283.239321,124.029999,11.022804
min,-10404.730000,-128718.852700,-20854.566238,-6594.509202,-7.671466e+06,-1.800064e+07,-176188.604575,-48955.437468,-6822.031204
25%,-571.530000,-372.910230,-232.132510,-34.473030,-3.201238e+05,4.184159e+02,0.001890,0.039139,0.038689
50%,-1.660000,-0.021319,-3.577480,0.000000,8.672689e+02,5.570117e+02,0.402684,0.344181,0.058396
75%,564.370000,335.336937,214.166184,33.112253,3.052859e+05,7.381082e+02,1.511817,0.800240,0.094521
max,10664.380000,72387.984792,26475.954120,2430.076459,7.822484e+06,2.451540e+07,60355.442914,69533.022803,5624.547003


### Why are MEFs negative? 

In [32]:
df['year'] = pd.DatetimeIndex(df.reset_index().DATE_UTC).year
df = df.loc[df['isorto'] == 'PJM']
df.loc[df['year']==2017]
df.head()

,isorto,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,co2_mef,so2_mef,nox_mef,pm25_mef,year
DATE_UTC,,,,,,,,,,,
2006-01-01 09:00:00,PJM,-808.0,-6289.007267,-870.852188,-153.495702,-753054.268500,931.997857,7.783425,1.077787,0.189970,2006
2006-01-01 10:00:00,PJM,-4.0,-1804.548640,13.971541,78.743658,-62142.172500,15535.543125,451.137160,-3.492885,-19.685915,2006
2006-01-01 11:00:00,PJM,702.0,-631.334293,705.651260,160.027434,255917.795685,364.555265,-0.899337,1.005201,0.227959,2006
2006-01-01 12:00:00,PJM,495.5,3863.412254,2233.035684,162.204678,376440.951675,759.719378,7.796997,4.506631,0.327356,2006
2006-01-01 13:00:00,PJM,803.5,2245.861451,1257.627818,158.666657,649538.109705,808.385949,2.795098,1.565187,0.197469,2006


In [33]:
df_pjm = df.loc[df['isorto'] == 'PJM']

# Keep only negative MEFs
mef_columns = ['co2_mef', 'so2_mef', 'nox_mef', 'pm25_mef']
df_pjm_neg = df_pjm[(df_pjm[mef_columns]<0).any(axis='columns')]

# Look at only 2017 data to compare with PJM data miner 
df_pjm_neg = df_pjm_neg.loc[df_pjm_neg['year']==2017]

df_pjm_neg

,isorto,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,co2_mef,so2_mef,nox_mef,pm25_mef,year
DATE_UTC,,,,,,,,,,,
2017-01-01 00:00:00,PJM,-345.67,978.663749,-318.302743,24.493995,-8.134274e+04,235.319070,-2.831208,0.920828,-0.070859,2017
2017-01-01 01:00:00,PJM,-1433.47,26.728816,-728.121301,-86.182575,-9.157642e+05,638.844377,-0.018646,0.507943,0.060122,2017
2017-01-01 13:00:00,PJM,649.87,-329.797218,388.223043,47.355057,4.421747e+05,680.404803,-0.507482,0.597386,0.072869,2017
2017-01-01 19:00:00,PJM,-352.71,-1354.056488,292.249779,-19.413759,-4.175773e+05,1183.911019,3.839008,-0.828584,0.055042,2017
2017-01-01 20:00:00,PJM,-96.62,-485.576133,390.376244,-7.166762,-2.535201e+05,2623.888488,5.025628,-4.040325,0.074175,2017
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 17:00:00,PJM,-858.20,-956.669526,99.708593,-120.020576,-4.317248e+05,503.058501,1.114740,-0.116183,0.139852,2017
2017-12-31 19:00:00,PJM,880.47,-47431.737144,1445.617662,-5.261673,3.867937e+05,439.303721,-53.870929,1.641870,-0.005976,2017
2017-12-31 20:00:00,PJM,7.56,84.128162,-1084.606964,-14.787116,1.051355e+05,13906.810056,11.128064,-143.466530,-1.955968,2017


A negative MEF means that either: 
- Generation decreased while emissions increased 
    - is there a situation where this can happen?
- Generation increased while emissions decreased 
    - perhaps generation shifted from coal to gas 

In [34]:
# Percent where generation increased while emission decreased 
df_pjm_neg.gload_mwh[df_pjm_neg.gload_mwh > 0].count() / df_pjm_neg.gload_mwh.count()

0.49636363636363634

##### Are MEFs negative because generation shifted from coal to gas? 

In [35]:
# Get data from PJM data miner 
df_pjm_gen = pd.read_csv("pjm_data/pjm_gen.csv", index_col='datetime_beginning_ept')
df_pjm_gen.index = pd.to_datetime(df_pjm_gen.index)
del df_pjm_gen['datetime_beginning_utc']
del df_pjm_gen['is_renewable']

# Keep only fossil fuel rows 
fossil_fuel = ['Coal', 'Gas', 'Oil']
df_pjm_gen = df_pjm_gen.loc[df_pjm_gen['fuel_type'].isin(fossil_fuel)]
df_pjm_gen.head()

,fuel_type,mw,fuel_percentage_of_total
datetime_beginning_ept,,,
2017-01-01 00:00:00,Coal,33866.5,0.41
2017-01-01 00:00:00,Gas,10516.4,0.13
2017-01-01 00:00:00,Oil,175.7,0.00
2017-01-01 01:00:00,Coal,33164.4,0.41
2017-01-01 01:00:00,Gas,10203.0,0.13


In [36]:
# Get dates where MEFs are negative and generation increased 
df_neg_increased_gen = df_pjm_neg[df_pjm_neg.gload_mwh > 0]
neg_dates = df_neg_increased_gen.index.values
yesterday = lambda x: pd.to_datetime(x) - timedelta(hours=1) 
yesterday_arr = yesterday(neg_dates)
yesterday_arr = np.delete(yesterday_arr, 0) # remove first elem as it's in 2016


# Show dates with negative index 
gen_mix_df = df_pjm_gen[df_pjm_gen.index.isin(df_neg_increased_gen.index) | df_pjm_gen.index.isin(yesterday_arr)]

# save as csv 
gen_mix_df.to_csv('gen_mix.csv')

gen_mix_df

,fuel_type,mw,fuel_percentage_of_total
datetime_beginning_ept,,,
2017-01-01 13:00:00,Coal,32187.6,0.40
2017-01-01 13:00:00,Gas,11058.5,0.14
2017-01-01 13:00:00,Oil,175.0,0.00
2017-01-01 20:00:00,Coal,35169.3,0.39
2017-01-01 20:00:00,Gas,15374.2,0.17
...,...,...,...
2017-12-30 20:00:00,Gas,24769.9,0.21
2017-12-30 20:00:00,Oil,1808.9,0.02
2017-12-30 21:00:00,Coal,47927.3,0.42


##### Check if generation data from CEMS matches data from PJM 

In [37]:
# Get total generation data from CEMS
df_isorto = pd.read_csv("../implement-emissions-assumptions/data/formatted_data/cems_isorto.csv", index_col='DATE_UTC')
df_isorto.index = pd.DatetimeIndex(df_isorto.reset_index().DATE_UTC) + timedelta(hours=5)
df_isorto = df_isorto.loc[df_isorto['isorto'] == 'PJM']
df_isorto = df_isorto.loc['2017-01-01':'2017-12-31']

In [38]:
# Find total fossil fuel generation at each time 
df_pjm_gen = df_pjm_gen[['mw']]
df_pjm_gen = df_pjm_gen.sum(level='datetime_beginning_ept')

# Dataframe to compare PJM's published total with CEMS published totals 
df_isorto = df_isorto[['gload_mwh']]
df_compare = pd.merge(df_isorto,df_pjm_gen, how='inner', left_index=True, right_index=True)
df_compare.columns = ['cems', 'pjm_data_miner']

# find difference between two columns, where calculated MEF is negative 
df_compare = df_compare.diff()
df_compare = df_compare.loc[(df_compare.index).isin(df_pjm.index)]
df_compare

,cems,pjm_data_miner
2017-01-01 00:00:00,NaN,NaN
2017-01-01 01:00:00,-1433.47,-1013.9
2017-01-01 02:00:00,-2531.72,-1458.5
2017-01-01 03:00:00,-2472.49,-686.2
2017-01-01 04:00:00,-2321.17,24.1
...,...,...
2017-12-30 20:00:00,699.96,-220.9
2017-12-30 21:00:00,1349.93,-1223.0
2017-12-30 22:00:00,3973.47,-1781.7
2017-12-30 23:00:00,1839.13,-1164.0


### Compare with PJM's Published MEFs 

In [39]:
df_pjm = df_pjm.loc['2014-01-01':'2018-12-31']

# Keep only MEF cols 
df_pjm = df_pjm[mef_columns]

# convert to lbs/MWh
LBS_CONVERSION = 2.20462
df_pjm = df_pjm.applymap(lambda x: x * LBS_CONVERSION)

# Aggregate over month
df_pjm['year'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).year
df_pjm['month'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).month
group_by = df_pjm.groupby([(df_pjm['year']),(df_pjm['month'])]).mean()

group_by

co2_mef    so2_mef    nox_mef  pm25_mef  year  month
year month                                                          
2014 1      1524.858319   0.154153   0.939685  0.105300  2014      1
     2      1402.895206   2.929072   0.966043  0.190460  2014      2
     3      1808.642317   1.144955   1.899357  0.204651  2014      3
     4      3021.934188   2.548116   6.120119  0.116401  2014      4
     5      1163.368044   0.971269   0.523640  0.096302  2014      5
     6      1369.913979   1.853775   1.095310  0.183909  2014      6
     7      1530.170236   1.519082   0.611457  0.188622  2014      7
     8      3174.174295  20.847987   4.096390  0.443954  2014      8
     9      -458.668066 -15.412292  -3.524204  0.137363  2014      9
     10     1442.801143   1.191122  -0.756360  0.186829  2014     10
     11     1297.462159   2.404767   1.886100  0.194435  2014     11
     12     1135.532850   1.258169   0.702501  0.165042  2014     12
2015 1       314.294354  -8.226740  -1.655048  0.064174  2015      1
     2      1442.906253   0.971043   1.916910  0.188970  2015      2
     3      1308.641595   2.255890   0.660841  0.217102  2015      3
     4      1244.561700   4.344137   2.500557  0.105797  2015      4
     5      1639.443686   3.659587   1.784576  0.153853  2015      5
     6      1759.966188   1.077088   1.190211  0.247294  2015      6
     7      1467.778108   2.172279   1.162161  0.188764  2015      7
     8      1536.322237   2.342105   1.547283  0.180123  2015      8
     9      1393.410636   0.619963   1.159863  0.178663  2015      9
     10     1484.959839   3.126105   1.204958  0.192912  2015     10
     11     1580.359797   2.246036   1.629487  0.237546  2015     11
     12     1786.101638   5.351047   1.723239  0.167315  2015     12
2016 1       977.940915   7.067388  -1.537521  0.660593  2016      1
     2      1482.682871   1.883297   1.294680  0.171166  2016      2
     3      1464.924951   3.289655   1.678965  0.230744  2016      3
     4      1593.240604   1.637767   1.218340  0.172460  2016      4
     5      1367.415536   2.545828   1.137084  0.161442  2016      5
     6      1375.223925   1.445327   1.061298  0.171802  2016      6
     7      1794.789746   1.038651  -0.426063  0.199152  2016      7
     8      1505.754534   1.180871   0.542681  0.173135  2016      8
     9      1454.035500   0.818838   0.633458  0.170029  2016      9
     10     1449.365460   0.971296   1.344711  0.199774  2016     10
     11     1629.757671   2.301339   1.915913  0.201712  2016     11
     12     3728.808818   5.365655  21.880258  0.476603  2016     12
2017 1     -9627.041409 -18.878201 -48.846697 -1.440128  2017      1
     2      1954.983731   2.577424   2.881514  0.252932  2017      2
     3      1779.397125  -0.748472  -0.378222  0.203289  2017      3
     4      2004.041548   2.777038   3.420609  0.356007  2017      4
     5       281.602841   4.573205  -2.561605 -0.057536  2017      5
     6      1425.892278  -1.039946  -1.177900  0.406120  2017      6
     7      1403.835192   0.904119   1.358407  0.177218  2017      7
     8       893.370551   1.929426   0.911444  0.110049  2017      8
     9      1337.509178   0.412678   1.030645  0.157525  2017      9
     10     1582.502339   1.802316   1.057427  0.185060  2017     10
     11     1479.863547  -1.880224   0.468642  0.067568  2017     11
     12     1522.880220   1.582153   0.994359  0.177548  2017     12
2018 1      1313.101546  -0.327752   0.616085  0.141977  2018      1

### Try removing values 3 standard deviations away 

In [40]:
# Remove outliers 
df_noOutliers = df[np.abs(df.co2_mef-df.co2_mef.mean()) <= (3*df.co2_mef.std())]

In [41]:
df_noOutliers[mef_columns].describe()

,co2_mef,so2_mef,nox_mef,pm25_mef
count,105156.000000,105156.000000,105156.000000,105156.000000
mean,697.852531,2.202143,0.732861,0.137459
std,2969.612607,54.899212,20.472214,1.904812
min,-158885.615811,-11145.934779,-3306.603601,-301.099021
25%,595.087699,0.699021,0.352295,0.079632
50%,692.491623,1.735040,0.685207,0.118067
75%,784.972612,3.528198,1.123777,0.208187
max,173621.252307,4874.763304,2680.766456,316.002775


In [42]:
# Find monthly MEF for PJM (to compare with their published data )
df_noOutlier_pjm = df_noOutliers.loc[df_noOutliers['isorto'] == 'PJM']
df_noOutlier_pjm = df_noOutlier_pjm.loc['2014-01-01':'2018-12-31']

# Keep MEF columns only
df_noOutlier_pjm = df_noOutlier_pjm[mef_columns]

# convert to lbs/MWh
LBS_CONVERSION = 2.20462
df_noOutlier_pjm = df_noOutlier_pjm.applymap(lambda x: x * LBS_CONVERSION)

# Year and month 
df_noOutlier_pjm['year'] = pd.DatetimeIndex(df_noOutlier_pjm.reset_index().DATE_UTC).year
df_noOutlier_pjm['month'] = pd.DatetimeIndex(df_noOutlier_pjm.reset_index().DATE_UTC).month

# Aggregate over months 
group_by = df_noOutlier_pjm.groupby([(df_noOutlier_pjm['year']),(df_noOutlier_pjm['month'])]).mean()

group_by

co2_mef   so2_mef   nox_mef  pm25_mef  year  month
year month                                                        
2014 1      1524.858319  0.154153  0.939685  0.105300  2014      1
     2      1402.895206  2.929072  0.966043  0.190460  2014      2
     3      1808.642317  1.144955  1.899357  0.204651  2014      3
     4      1771.781748  3.070639  2.088509  0.200012  2014      4
     5      1739.809923  4.130349  2.011440  0.226340  2014      5
     6      1369.913979  1.853775  1.095310  0.183909  2014      6
     7      1530.170236  1.519082  0.611457  0.188622  2014      7
     8      1567.546105  2.577871  1.611748  0.195268  2014      8
     9      1717.402121  2.981439  1.037930  0.228590  2014      9
     10     1442.801143  1.191122 -0.756360  0.186829  2014     10
     11     1297.462159  2.404767  1.886100  0.194435  2014     11
     12     1135.532850  1.258169  0.702501  0.165042  2014     12
2015 1       973.447718 -1.516929  0.522344  0.175405  2015      1
     2      1442.906253  0.971043  1.916910  0.188970  2015      2
     3      1308.641595  2.255890  0.660841  0.217102  2015      3
     4      1244.561700  4.344137  2.500557  0.105797  2015      4
     5      1639.443686  3.659587  1.784576  0.153853  2015      5
     6      1759.966188  1.077088  1.190211  0.247294  2015      6
     7      1467.778108  2.172279  1.162161  0.188764  2015      7
     8      1536.322237  2.342105  1.547283  0.180123  2015      8
     9      1393.410636  0.619963  1.159863  0.178663  2015      9
     10     1484.959839  3.126105  1.204958  0.192912  2015     10
     11     1580.359797  2.246036  1.629487  0.237546  2015     11
     12     1786.101638  5.351047  1.723239  0.167315  2015     12
2016 1      1727.561525  4.237343  1.154640  0.237936  2016      1
     2      1482.682871  1.883297  1.294680  0.171166  2016      2
     3      1464.924951  3.289655  1.678965  0.230744  2016      3
     4      1593.240604  1.637767  1.218340  0.172460  2016      4
     5      1367.415536  2.545828  1.137084  0.161442  2016      5
     6      1375.223925  1.445327  1.061298  0.171802  2016      6
     7      1794.789746  1.038651 -0.426063  0.199152  2016      7
     8      1505.754534  1.180871  0.542681  0.173135  2016      8
     9      1454.035500  0.818838  0.633458  0.170029  2016      9
     10     1449.365460  0.971296  1.344711  0.199774  2016     10
     11     1629.757671  2.301339  1.915913  0.201712  2016     11
     12      980.680839  2.722553  0.839231  0.190810  2016     12
2017 1      1258.280558  0.593083  0.605938  0.146089  2017      1
     2      1954.983731  2.577424  2.881514  0.252932  2017      2
     3      1065.127306  2.151295  0.392809  0.112517  2017      3
     4      2004.041548  2.777038  3.420609  0.356007  2017      4
     5      1507.141063  2.268854  1.002665  0.150340  2017      5
     6      1425.892278 -1.039946 -1.177900  0.406120  2017      6
     7      1403.835192  0.904119  1.358407  0.177218  2017      7
     8      1421.424790  1.058013  0.571089  0.176456  2017      8
     9      1337.509178  0.412678  1.030645  0.157525  2017      9
     10     1582.502339  1.802316  1.057427  0.185060  2017     10
     11     1479.863547 -1.880224  0.468642  0.067568  2017     11
     12     1522.880220  1.582153  0.994359  0.177548  2017     12
2018 1      1313.101546 -0.327752  0.616085  0.141977  2018      1